In [1]:
%run init_notebook.py


In [2]:
from src.utils import load_pd_df, get_dt_index
from src.processing import pd_join_dfs, pd_groupby, xcorr
from src.statsmodels import *
import scipy

In [3]:
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.simplefilter('ignore', ValueWarning)

In [4]:
from src.utils import load_pd_df, save_pd_df, get_dt_index, pd_join_freq, Capturing, cross_corr, pd_df_astype, save_fig, get_stars
from src.processing import pd_groupby, pd_join_dfs, adf_test_summary, hausman, plt_stacked_bar, get_individual_perc_error
from src.pymc_modelling import get_samp

from statsmodels.stats.diagnostic import het_white, het_breuschpagan
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson

from linearmodels.panel import PanelOLS
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.arima.model import ARIMA
from linearmodels import RandomEffects
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.vector_ar.svar_model import SVAR
from statsmodels.tsa.ar_model import ar_select_order
from statsmodels.tsa.vector_ar.vecm import select_order

from statsmodels.iolib.summary2 import summary_col

In [5]:
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.vector_ar.vecm import VECM, select_coint_rank, select_order
from statsmodels.tsa.stattools import adfuller

In [8]:
df_micro = load_pd_df("df_analysis_micro.feather")

In [9]:
DICT_PARSE_COLS.update(
    {
        'T_pca_sum': float,
        'T_pca_sum_diff': float,
        'pi_de_surprise_Y': float,
        'pi_de_estimate_Y': float,
        'delta_pe_error': float,
        'hhinc_midpoint': 'categoryO',
    }
)

In [10]:
df_micro = pd_df_astype(df_micro, DICT_PARSE_COLS)

## panel structure sparsity and distance analysis

In [11]:
ind_cols = ['id', 'week_recorded']
sub = df[['pi_perc'] + ind_cols].copy().reset_index(drop=True).drop_duplicates()

filt = sub.groupby(ind_cols).pi_perc.last().unstack().count(axis=1) > 7
sub = sub.loc[sub.id.isin(list(filt[filt].index))].set_index(ind_cols).sort_index()
sub = sub.dropna()
filt.sum()

377

In [12]:
dist = df.groupby('id')[
    ['hhinc_midpoint', 'pi_perc_error', 'debt', 'wealth_bank', 'savings_planned', 'eduschool', 'hhsize', 'ecbtrust']
                       ].last().astype(float)

dist = dist.loc[sub.index.get_level_values(0)]
dist = dist.loc[~dist.index.duplicated()]
dist = scipy.spatial.distance.cdist(dist, dist)

dist[np.isnan(dist)] = np.array(list([0] * np.isnan(dist).sum()))
dist = dist.mean(axis=0)

KeyError: "Columns not found: 'pi_perc_error'"

In [ ]:
A = ~sub.unstack().isna()
A = (A * ((dist - dist.min()) / (dist.max() - dist.min()))[:, None]).replace({0: np.nan})
A.columns = A.columns.get_level_values(1)

# fig = plt.figure(figsize=(16,5))
fig, ax = get_fig_subplots()
fig.suptitle('Distribution of respondents over time, Euclidean distance from mean respondent in shades')
fig.tight_layout()
sns.heatmap(A, xticklabels=False, yticklabels=False, ax=ax, cmap='winter')
save_fig(fig, "pols_dist_matrix.png")

## POLS: How do news affect inflation perception on an individual level?

Investigate the effect of news $N_t$ on inflation perception $\pi^p_{i,t}$:
$$ \pi^p_{i,t} = \alpha_i + \beta N_t + \gamma X_{i,t} + \epsilon_{i,t} $$

In [15]:
cols = ['T_sum_diff_lag', 'hhinc_midpoint', 'pi_de_surprise_Y', 'pi_de_estimate_Y', 'is_unempl',]  #'debt',]
ind_cols = ['id', 'week_recorded']
dep_col = ['pi_perc']

In [16]:
sub = df_micro[ind_cols + cols + dep_col].copy().reset_index(drop=True)
filt = sub.groupby(ind_cols)[dep_col + cols].last()
filt = filt.dropna().reset_index().groupby('id').count().min(axis=1) > 5
sub = sub.loc[sub.id.isin(set(filt[filt].index))].set_index(ind_cols).sort_index().dropna()
sub = pd_df_astype(sub)

sub.shape, filt.sum()

((2960, 6), 473)

In [17]:
m_fe1 = PanelOLS(sub[dep_col], sub[cols], entity_effects=True, ).fit(cov_type='clustered', cluster_entity=True)
m_re = RandomEffects(sub[dep_col], sub[cols]).fit()

In [18]:
dist, filt = get_cooks_distance(sub[cols].values.astype(float), m_fe1.resids.values, flt_largest_perc=90.)
sub2 = sub.drop(sub.loc[filt].index.get_level_values(0))
m_fe12 = PanelOLS(sub2[dep_col], sub2[cols],
                   entity_effects=True, drop_absorbed=True, check_rank=False).fit(cov_type="clustered", cluster_entity=True)

C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_3404\3742255664.py:4: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

hhinc_midpoint.12000.0

  entity_effects=True, drop_absorbed=True, check_rank=False).fit(cov_type="clustered", cluster_entity=True)


In [19]:
out = get_statsmodels_summary([m_fe1, m_fe12], ['print'], seperator=" ", is_filt_sig=True)
out

print                     
                                    pi_perc            pi_perc_1
T_sum_diff_lag           -0.287 *** [-3.86]   -0.128 ** [-2.166]
hhinc_midpoint.2250.0       -0.01  [-0.679]   -0.024 ** [-1.969]
is_unempl                  -0.007  [-0.735]   -0.008 ** [-2.292]
pi_de_estimate_Y         1.619 *** [46.332]   1.707 *** [64.431]
pi_de_surprise_Y       -0.648 *** [-18.329]  -0.65 *** [-26.529]
N                                    2960.0               1797.0
R^2                                   0.496                0.742
R^2 between                           0.793                0.671

In [20]:
# plt.scatter(m_fe.predict().values, m_fe.resids.values)

In [21]:
# H0: Homoscedasticity is present
_ = sm.add_constant(pd.concat([m_fe1.resids, sub[cols]], axis=1))
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, het_breuschpagan(_[['residual']], _.drop('residual', axis=1)))))

{'LM-Stat': 9.731528055959817, 'LM p-val': 0.08320964876006931, 'F-Stat': 1.9487673173256976, 'F p-val': 0.08317198492149712}


In [22]:
durbin_watson_test_results = durbin_watson(m_fe1.resids) 
print(durbin_watson_test_results)

2.0038560253492794


In [23]:
# H0: RE is to be preferred
hausman_results = hausman(m_fe1, m_re)
print('chi-Squared: ' + str(hausman_results[0]))
print('degrees of freedom: ' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

chi-Squared: 14.719913142667025
degrees of freedom: 16
p-Value: 0.545240206035982


# Exp 2

Influence of news on future change in inflation $\pi^e_{i,t} - \pi^p_{i,t}$
$$ \pi^e_{i,t} - \pi^p_{i,t} = \alpha_i + \beta N_t + \gamma X_{i,t} + \epsilon_{i,t} $$

In [24]:
cols = ['T_sum_diff_lag', 'hhinc_midpoint', 'pi_de_surprise_Y', 'pi_de_estimate_Y', 'debt', 'is_unempl', 'wealth_bank',]  #'debt',]
ind_cols = ['id', 'week_recorded']
dep_col = ['delta_pe']

In [25]:
sub = df_micro[ind_cols + cols + dep_col].copy().reset_index(drop=True)
filt = sub.groupby(ind_cols)[dep_col + cols].last()
filt = filt.dropna().reset_index().groupby('id').count().min(axis=1) > 3
sub = sub.loc[sub.id.isin(set(filt[filt].index))].set_index(ind_cols).sort_index().dropna()
sub = pd_df_astype(sub)
sub.shape, filt.sum()

((2982, 8), 715)

In [26]:
m_fe2 = PanelOLS(sub[dep_col], sub[cols], entity_effects=True).fit(cov_type='clustered', cluster_entity=True)
m_re = RandomEffects(sub[dep_col], sub[cols]).fit()
# print(m_fe2)

In [27]:
dist, filt = get_cooks_distance(sub[cols].values.astype(float), m_fe2.resids.values, flt_largest_perc=80.)
sub2 = sub.drop(sub.loc[filt].index.get_level_values(0))
m_fe22 = PanelOLS(sub2[dep_col], sub2[cols],
                   entity_effects=True, drop_absorbed=True, check_rank=False).fit(cov_type="clustered", cluster_entity=True)

C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_3404\3177373305.py:4: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

debt.9.0

  entity_effects=True, drop_absorbed=True, check_rank=False).fit(cov_type="clustered", cluster_entity=True)


In [28]:
get_statsmodels_summary([m_fe2, m_fe22], ['print'], seperator=" ", is_filt_sig=True)

print                      
                                     delta_pe            delta_pe_1
hhinc_midpoint.2750.0        0.009 ** [2.452]     0.009 *** [8.838]
is_unempl                  -0.004 ** [-2.202]   -0.004 *** [-3.118]
hhinc_midpoint.9000.0       0.011 *** [2.659]     0.011 *** [6.391]
hhinc_midpoint.750.0           0.007 * [1.88]     0.014 *** [5.621]
hhinc_midpoint.7000.0       0.011 *** [2.965]      0.011 *** [6.98]
hhinc_midpoint.5500.0        0.01 *** [2.764]     0.011 *** [7.304]
hhinc_midpoint.4250.0       0.011 *** [2.953]      0.011 *** [9.04]
hhinc_midpoint.3750.0        0.009 ** [2.559]      0.01 *** [8.028]
hhinc_midpoint.3250.0       0.011 *** [2.959]     0.011 *** [9.887]
pi_de_estimate_Y           0.706 *** [79.958]    0.759 *** [67.353]
pi_de_surprise_Y        -0.756 *** [-104.121]  -0.797 *** [-89.998]
hhinc_midpoint.1750.0        0.008 ** [2.343]     0.011 *** [10.79]
hhinc_midpoint.1250.0        0.009 ** [2.446]    0.012 *** [12.125]
hhinc_midpoint.12000.0         0.01 ** [2.12]     0.006 *** [3.723]
debt.8.0                       0.001  [0.893]     0.004 *** [2.662]
debt.4.0                   -0.002 ** [-2.211]       -0.001  [-1.31]
T_sum_diff_lag             2.217 *** [37.911]     2.536 *** [33.03]
hhinc_midpoint.2250.0        0.008 ** [2.336]    0.009 *** [11.202]
R^2 between                            -5.578                -6.771
R^2                                     0.785                 0.898
N                                      2982.0                1275.0

In [29]:
# H0: Homoscedasticity is present
_ = sm.add_constant(pd.concat([m_fe2.resids, sub[cols]], axis=1))
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, het_breuschpagan(_[['residual']], _.drop('residual', axis=1)))))

{'LM-Stat': 43.181886376026625, 'LM p-val': 3.076435191493403e-07, 'F-Stat': 6.242690823174869, 'F p-val': 2.77045765767781e-07}


In [30]:
durbin_watson_test_results = durbin_watson(m_fe2.resids) 
print(durbin_watson_test_results)

2.3842923018574034


In [31]:
# H0: RE is to be preferred
hausman_results = hausman(m_fe2, m_re)
print('chi-Squared: ' + str(hausman_results[0]))
print('degrees of freedom: ' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

chi-Squared: 41.04889561436247
degrees of freedom: 34
p-Value: 0.18898709599459806


# Exp 3

Influence of news on expectation:
$$ \pi^e_{i,t} = \alpha_i + \beta N_t + \gamma X_{i,t} + \epsilon_{i,t} $$

In [32]:
# cols = ['T_sum_diff_lag', 'hhinc_midpoint', 'pi_de_Y',]# 'debt',] # 'is_unempl', ] #'wealth_bank'] #, 'eduwork']
cols = ['T_sum_diff_lag', 'hhinc_midpoint', 'pi_de_surprise_Y', 'pi_de_estimate_Y', ]
ind_cols = ['id', 'week_recorded']
dep_col = ['pi_exp']

In [33]:
sub = df_micro[ind_cols + cols + dep_col].copy().reset_index(drop=True)
filt = sub.groupby(ind_cols)[dep_col + cols].last()
filt = filt.dropna().reset_index().groupby('id').count().min(axis=1) > 5
sub = sub.loc[sub.id.isin(set(filt[filt].index))].set_index(ind_cols).sort_index().dropna()
sub = pd_df_astype(sub)
sub.shape, filt.sum()

((29619, 5), 4300)

In [34]:
m_fe3 = PanelOLS(sub[dep_col], sub[cols], entity_effects=True).fit(cov_type='clustered', cluster_entity=True)
m_re = RandomEffects(sub[dep_col], sub[cols]).fit()

In [35]:
dist, filt = get_cooks_distance(sub[cols].values.astype(float), m_fe3.resids.values, flt_largest_perc=90.)
sub2 = sub.drop(sub.loc[filt].index.get_level_values(0))
m_fe32 = PanelOLS(sub2[dep_col], sub2[cols],
                   entity_effects=True, drop_absorbed=True, check_rank=False).fit(cov_type="clustered", cluster_entity=True)

In [36]:
get_statsmodels_summary([m_fe3, m_fe32], ['print'], seperator=" ", is_filt_sig=True)

print                     
                                    pi_exp             pi_exp_1
T_sum_diff_lag         -0.199 *** [-6.784]   -0.205 *** [-8.93]
hhinc_midpoint.2250.0     -0.003  [-0.805]   -0.005 ** [-2.253]
hhinc_midpoint.2750.0      -0.003  [-0.82]   -0.005 ** [-2.561]
hhinc_midpoint.3250.0     -0.003  [-0.706]   -0.005 ** [-2.201]
hhinc_midpoint.3750.0     -0.002  [-0.595]   -0.004 ** [-1.972]
hhinc_midpoint.4250.0     -0.004  [-0.854]   -0.005 ** [-2.229]
hhinc_midpoint.5500.0     -0.003  [-0.819]   -0.005 ** [-2.134]
hhinc_midpoint.750.0      -0.003  [-0.601]  -0.006 *** [-2.581]
pi_de_estimate_Y        0.983 *** [72.192]   1.089 *** [99.872]
pi_de_surprise_Y         0.18 *** [13.707]    0.133 *** [12.84]
N                                  29619.0              16826.0
R^2                                  0.444                 0.68
R^2 between                          0.808                0.846

In [37]:
# H0: Homoscedasticity is present
_ = sm.add_constant(pd.concat([m_fe3.resids, sub[cols]], axis=1))
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, het_breuschpagan(_[['residual']], _.drop('residual', axis=1)))))

{'LM-Stat': 133.96010088206506, 'LM p-val': 5.5375183496182e-28, 'F-Stat': 33.636502113400134, 'F p-val': 4.810430376888195e-28}


In [38]:
durbin_watson_test_results = durbin_watson(m_fe3.resids) 
print(durbin_watson_test_results)

1.9851732576410788


In [39]:
# H0: RE is to be preferred
hausman_results = hausman(m_fe3, m_re)
print('chi-Squared: ' + str(hausman_results[0]))
print('degrees of freedom: ' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

chi-Squared: 52.75638146924047
degrees of freedom: 15
p-Value: 4.232641067169811e-06


## Exp 4

In [40]:
cols = ['T_sum_diff_lag', 'hhinc_midpoint', 'pi_de_surprise_Y', 'pi_de_estimate_Y', ] #'debt', 'is_unempl', 'wealth_bank',]  #'debt',]
ind_cols = ['id', 'week_recorded']
dep_col = ['pi_perc_error']

In [41]:
sub = df_micro[ind_cols + cols + dep_col].copy().reset_index(drop=True)
filt = sub.groupby(ind_cols)[dep_col + cols].last()
filt = filt.dropna().reset_index().groupby('id').count().min(axis=1) > 4
sub = sub.loc[sub.id.isin(set(filt[filt].index))].set_index(ind_cols).sort_index().dropna()
sub.shape, filt.sum()

((8055, 5), 1492)

In [42]:
m_fe4 = PanelOLS(sub[dep_col], sub[cols], entity_effects=True).fit(cov_type='clustered', cluster_entity=True)
m_re = RandomEffects(sub[dep_col], sub[cols]).fit()

In [43]:
# plt.scatter(m_fe4.predict().values, m_fe4.resids.values)

In [44]:
# H0: Homoscedasticity is present
_ = sm.add_constant(pd.concat([m_fe4.resids, sub[cols]], axis=1))
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, het_breuschpagan(_[['residual']], _.drop('residual', axis=1)))))

{'LM-Stat': 53.327093922171194, 'LM p-val': 7.279080972342227e-11, 'F-Stat': 13.412292376616799, 'F p-val': 6.769040249195835e-11}


In [45]:
durbin_watson_test_results = durbin_watson(m_fe4.resids) 
print(durbin_watson_test_results)

2.1854819389607507


In [46]:
# H0: RE is to be preferred
hausman_results = hausman(m_fe4, m_re)
print('chi-Squared: ' + str(hausman_results[0]))
print('degrees of freedom: ' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

chi-Squared: 19.294110136226617
degrees of freedom: 15
p-Value: 0.20071428682485978


## Exp 5

In [47]:
cols = ['T_sum_diff_lag', 'hhinc_midpoint', 'pi_de_surprise_Y', 'pi_de_estimate_Y', ] #'debt', 'is_unempl', 'wealth_bank',]  #'debt',]
ind_cols = ['id', 'week_recorded']
dep_col = ['delta_pe_error']

In [48]:
sub = df_micro[ind_cols + cols + dep_col].copy().reset_index(drop=True)
filt = sub.groupby(ind_cols)[dep_col + cols].last()
filt = filt.dropna().reset_index().groupby('id').count().min(axis=1) > 4
sub = sub.loc[sub.id.isin(set(filt[filt].index))].set_index(ind_cols).sort_index().dropna()
sub.shape, filt.sum()

((45794, 5), 7279)

In [49]:
sub = sub.loc[~sub.duplicated()]

In [50]:
m_fe5 = PanelOLS(sub[dep_col], sub[cols], entity_effects=True).fit(cov_type='clustered', cluster_entity=True)
m_re = RandomEffects(sub[dep_col], sub[cols]).fit()
# print(m_fe5)

In [51]:
# plt.scatter(m_fe.predict().values, m_fe.resids.values)

In [52]:
# H0: Homoscedasticity is present
_ = sm.add_constant(pd.concat([m_fe5.resids, sub[cols]], axis=1))
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, het_breuschpagan(_[['residual']], _.drop('residual', axis=1)))))

{'LM-Stat': 40.1161500548235, 'LM p-val': 4.0955041155242976e-08, 'F-Stat': 10.153400377756073, 'F p-val': 3.676614306987792e-08}


In [53]:
durbin_watson_test_results = durbin_watson(m_fe5.resids) 
print(durbin_watson_test_results)

2.3733566765306287


In [54]:
# H0: RE is to be preferred
hausman_results = hausman(m_fe5, m_re)
print('chi-Squared: ' + str(hausman_results[0]))
print('degrees of freedom: ' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

chi-Squared: 60.15484977939887
degrees of freedom: 15
p-Value: 2.3720837561939567e-07


## summary

In [55]:
lst_fes = [m_fe1, m_fe2, m_fe3, m_fe4, m_fe5]
out = get_statsmodels_summary(lst_fes, seperator=" ", is_filt_sig=True)
save_pd_df(out, 'tab_pols.csv', GRAPHS_DIR)

In [56]:
out

,delta_pe,delta_pe_error,pi_exp,pi_perc,pi_perc_error
hhinc_midpoint.2750.0,0.009 ** [2.452],0.004 [0.937],-0.003 [-0.82],-0.006 [-0.447],0.002 [0.17]
is_unempl,-0.004 ** [-2.202],NaN,NaN,-0.007 [-0.735],NaN
hhinc_midpoint.9000.0,0.011 *** [2.659],0.002 [0.556],-0.002 [-0.416],-0.004 [-0.261],0.007 [0.664]
hhinc_midpoint.7000.0,0.011 *** [2.965],0.007 * [1.798],-0.003 [-0.699],-0.004 [-0.283],0.004 [0.385]
hhinc_midpoint.5500.0,0.01 *** [2.764],0.007 * [1.682],-0.003 [-0.819],-0.006 [-0.404],0.005 [0.503]
hhinc_midpoint.4250.0,0.011 *** [2.953],0.007 [1.56],-0.004 [-0.854],-0.006 [-0.374],0.004 [0.364]
hhinc_midpoint.3750.0,0.009 ** [2.559],0.003 [0.762],-0.002 [-0.595],-0.004 [-0.279],0.002 [0.239]
hhinc_midpoint.3250.0,0.011 *** [2.959],0.004 [0.891],-0.003 [-0.706],-0.006 [-0.386],0.004 [0.41]
pi_de_estimate_Y,0.706 *** [79.958],-1.077 *** [-38.988],0.983 *** [72.192],1.619 *** [46.332],0.116 *** [4.783]
pi_de_surprise_Y,-0.756 *** [-104.121],-0.995 *** [-40.972],0.18 *** [13.707],-0.648 *** [-18.329],-1.156 *** [-52.196]
